# Image Scraping in Google Images using Selenium Chromedriver

#### This notebook takes help from the following github repository
#### https://github.com/akash-shastri/Chrome_image_scraper/blob/master/Chrome_Image_scraper.ipynb

In [13]:
DRIVER_PATH = "G:\Deep Learning HPE\Age Detection Dataset\chromedriver.exe"
#search_term = ["New born boy","School going boy","Adolescent boy","College going boy","Working adult","Middle aged man","Old man"]
search_term = ["Indian college boy","American college boy","African college boy","British college boy"]
number_images = 100
target_path = './images'

In [14]:
from selenium import webdriver

In [15]:
def get_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [16]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")
        

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [17]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = get_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [18]:
import os 
import time
import requests
import io
from PIL import Image
import hashlib

for ele in search_term:
    search_and_download(search_term = ele, driver_path=DRIVER_PATH, target_path=target_path, number_images=number_images)

Found: 100 search results. Extracting links from 0:100
Found: 101 image links, done!
SUCCESS - saved https://as2.ftcdn.net/jpg/02/90/71/75/500_F_290717595_ZXzMKUrZNrHfeJRoFtcYVsAiBZgzFjJe.jpg - as ./images\indian_college_boy\5ee01fa3d2.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSadXzB4iRxPqulW3ysKyNKk9_Bz34u9yY1zA&usqp=CAU - as ./images\indian_college_boy\da3f500f0a.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR8WapkJnL4-sdnEjNciInoEKQJ1Iti7y7p6A&usqp=CAU - as ./images\indian_college_boy\db667787d2.jpg
SUCCESS - saved https://thumbs.dreamstime.com/z/handsome-college-student-28690181.jpg - as ./images\indian_college_boy\a47c5e140d.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSe4KA5LpOZowdFPqvhzLPSjJ5_L1xssgekxQ&usqp=CAU - as ./images\indian_college_boy\d56b1905d7.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR52KV635XIpM9RcDpHmeM4nXwKXoJGjLoVwA&usqp=CAU - as ./images\indi

SUCCESS - saved https://st3.depositphotos.com/5653638/16091/i/1600/depositphotos_160919446-stock-photo-handsome-indian-boy-or-male.jpg - as ./images\indian_college_boy\e4ca04251f.jpg
SUCCESS - saved https://i.pinimg.com/originals/09/ff/91/09ff91a85ea8f8afd81346f4a0a16aa7.jpg - as ./images\indian_college_boy\9c9c27d46d.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRiMrBR7NJGcL3KXpAeqCXNR_GtkZw48zq8CQ&usqp=CAU - as ./images\indian_college_boy\f441283027.jpg
SUCCESS - saved https://image.shutterstock.com/z/stock-photo-indian-college-boy-with-holding-bag-1464214523.jpg - as ./images\indian_college_boy\f64fcb4d1e.jpg
SUCCESS - saved https://res.cloudinary.com/twenty20/private_images/t_watermark-criss-cross-10/v1578193935000/photosp/b95f2250-f8b9-4c01-9db2-6ac5df52bc5e/stock-photo-smart-boy-indian-young-man-indian-college-student-b95f2250-f8b9-4c01-9db2-6ac5df52bc5e.jpg - as ./images\indian_college_boy\6935d54cf0.jpg
SUCCESS - saved https://static1.bigstockphoto.c

SUCCESS - saved https://as2.ftcdn.net/jpg/01/78/56/23/500_F_178562389_nsbcMZ8c4twIaPHHLieSuHhYaWaG6Chk.jpg - as ./images\american_college_boy\84e1397a6a.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS3nbL4F7C1RGgtP_CPd4SbDWKsCGqKx89TrQ&usqp=CAU - as ./images\american_college_boy\90ef4536a5.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQSJISE6YUmTfEsMHWJOBtu89NfIkRcW9SdXQ&usqp=CAU - as ./images\american_college_boy\d44dcd635c.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSZ70AXxI65PFaRjDoCyRIWg0qG9lM8l4AI_w&usqp=CAU - as ./images\american_college_boy\b534b9e189.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSAb8rj2lKXVSUj2m6NSpzLTNKLyWMR40x20Q&usqp=CAU - as ./images\american_college_boy\cff123b136.jpg
SUCCESS - saved https://i.pinimg.com/236x/b8/32/5c/b8325c8ada44cf6a9e4a7ee589593cbf.jpg - as ./images\american_college_boy\bf24aae99f.jpg
SUCCESS - saved https://encrypted-tbn0.gs

SUCCESS - saved https://hips.hearstapps.com/sev.h-cdn.co/assets/cm/15/09/54ee6dd82a43a_-_sev-eli-macdonald-usf-de.jpg - as ./images\american_college_boy\029c0205b5.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRqn6TfgHLBWmey8qeksOqoeMtLqN3Buy94-Q&usqp=CAU - as ./images\american_college_boy\d8d2f88980.jpg
SUCCESS - saved https://media.gettyimages.com/photos/african-american-teen-boy-using-smart-phone-to-text-at-high-school-picture-id900966302?s=612x612 - as ./images\american_college_boy\306e560c2b.jpg
SUCCESS - saved https://m.media-amazon.com/images/M/MV5BMTJmMjRiOGYtMTUzZC00N2UyLWJmOGEtM2I0MmM1ZTU2ZjZlL2ltYWdlL2ltYWdlXkEyXkFqcGdeQXVyNjk3MzQ0NjA@._V1_UY1200_CR285,0,630,1200_AL_.jpg - as ./images\american_college_boy\423bf4758a.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR9WTWg2qs6N7Q8BP-oSYDyDJTqcl7ijGC5_Q&usqp=CAU - as ./images\american_college_boy\ad992089ba.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:A

Found: 100 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTpWVgwD1p7VjBcibEnSnHTlR4TUcSOxGCgIw&usqp=CAU - as ./images\african_college_boy\8dfc2f7a9d.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRT3pUKJXO7rxK6K9cpifKGeCyvMFjDOKmTxw&usqp=CAU - as ./images\african_college_boy\5c8e7374ca.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRwJHTofW2SuyaqJJp34Jff0yYha2BOo_4DMw&usqp=CAU - as ./images\african_college_boy\212547444a.jpg
SUCCESS - saved https://st4.depositphotos.com/6179956/19711/i/1600/depositphotos_197118264-stock-photo-happy-african-college-girl-talking.jpg - as ./images\african_college_boy\698884b45c.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQuU_urZGgTXo1iY-w5HOjyAl91Hiww4wrPcQ&usqp=CAU - as ./images\african_college_boy\10a465a671.jpg
SUCCESS - saved https://image.shutterstock.com/image-photo/group-smiling-african-american-college-260nw-173134577.jpg -

C:\Users\Sanjay\anaconda3\lib\site-packages\PIL\Image.py:2942: UserWarning: image file could not be identified because WEBP support not installed
  warnings.warn(message)


ERROR - Could not save https://wallsheaven.com/decorating-ideas/200535675/home-design-living-room-sofa-apartament-200535675-african-college-students-using-laptop-together-C25553032.webp?nr=3&size=400 - cannot identify image file <_io.BytesIO object at 0x0000024AB0C96540>
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ2LlJ4jTR-ZTjikYvTNeHezK__OSffrp6hWA&usqp=CAU - as ./images\african_college_boy\8788adbe82.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS9MREA-JWqL_k-I0D-MN4iqHrKi4MvOLP9bw&usqp=CAU - as ./images\african_college_boy\bc4431aab7.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ2r-Wv8ktLxIkLpy0QxzJXAPZy6LXu9n482g&usqp=CAU - as ./images\african_college_boy\4a3fa598b1.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQV6DiwCs7oX25GikGoQFA56xLd8WlymQLIFw&usqp=CAU - as ./images\african_college_boy\878fc2d8b4.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9

SUCCESS - saved https://i.ytimg.com/vi/JaxtO5P3V2Q/maxresdefault.jpg - as ./images\african_college_boy\dcc804cac6.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRq7PcqXDuZ8qVFp7hS3EBpIrF8qmQRvsMgjQ&usqp=CAU - as ./images\african_college_boy\b6aae93b5b.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR7KgXjbhSY5wa_TydbjNhIUjRGBNZ-lwRdCw&usqp=CAU - as ./images\african_college_boy\cca157e68e.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSQ50UeAbxE2GXyoOMFTIJoWoO3RP2EYSECaw&usqp=CAU - as ./images\african_college_boy\874863dee2.jpg
SUCCESS - saved https://comps.canstockphoto.com/african-american-college-boy-on-campus-stock-images_csp26418753.jpg - as ./images\african_college_boy\d6312a64be.jpg
SUCCESS - saved https://thumbs.dreamstime.com/z/african-american-college-student-happy-boy-carrying-backpack-outdoors-campus-39084451.jpg - as ./images\african_college_boy\5626dfca08.jpg
SUCCESS - saved https://encrypted-t

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRuCnhypS9ht6RZYqNQblX8-rjG2U4NHCE-aw&usqp=CAU - as ./images\british_college_boy\4f456afb6a.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQKV6gE6MYNJnzdB8rKOJo_e8XA_zPRbdhqUA&usqp=CAU - as ./images\british_college_boy\7f930ca667.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQG9giQFccE3zqM7E3i2OpyPfI6ctqOJwuhMw&usqp=CAU - as ./images\british_college_boy\48653255a4.jpg
SUCCESS - saved https://img.nordangliaeducation.com/resources/us/_filecache/256/313/31774-padded-w600-h315-of-1-FFFFFF-copy-of-the_british_college_of_brazil_high-res-6279.jpg - as ./images\british_college_boy\d08e36b5d9.jpg
SUCCESS - saved https://d2yw9izrmawh7r.cloudfront.net/employers/1061757/brandimages/20181206_312PM_desktop_1061757_header.jpg - as ./images\british_college_boy\23a816bb93.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQu5dvuVOYXUNvarrJtsafiZun3IH9GZf8qDA

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRz6x6bRySvL6u5NUUow2-KnAz5QWbGFRueFw&usqp=CAU - as ./images\british_college_boy\c244b7b56e.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQE71JJLy2QBWp32hDa9Cbkt6-_PKurQGpPtQ&usqp=CAU - as ./images\british_college_boy\146a0a499c.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSJNZFinFrGetc5Rmv6FzoI0MmGqWejYmRZ2A&usqp=CAU - as ./images\british_college_boy\6dfa2a8b70.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRBE-J5mezyeaMh-4ux3_3dDcdh6GNcST_j8Q&usqp=CAU - as ./images\british_college_boy\febe94c07b.jpg
SUCCESS - saved https://media.gettyimages.com/photos/500th-anniversary-of-eton-college-british-independent-school-for-boys-picture-id618226526?s=612x612 - as ./images\british_college_boy\e2935c70cb.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRUzcuQn4B103keV077NEh717aCp9x92ljRYw&usqp=CAU - as ./images\british_